# NLP with BERT for Question Rewriting


### Importing the libraries

In [1]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 3.9 MB/s 
     |████████████████████████████████| 131 kB 34.3 MB/s 
     |████████████████████████████████| 8.5 MB 26.9 MB/s 
     |████████████████████████████████| 79 kB 1.7 MB/s 
     |████████████████████████████████| 138 kB 30.5 MB/s 
     |████████████████████████████████| 127 kB 29.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 39.6 MB/s 
     |████████████████████████████████| 3.3 MB 17.9 MB/s 
     |████████████████████████████████| 61 kB 468 kB/s 
     |████████████████████████████████| 895 kB 32.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import tensorflow as tf
import torch

from google.colab import files

import json
import pandas as pd
import numpy as np

from transformers import AutoTokenizer

from keras.preprocessing.sequence import pad_sequences

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from pytorch_pretrained_bert import BertModel, BertAdam

from tqdm import tqdm, trange
import matplotlib.pyplot as plt

### Retrieving the name of the GPU 

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla K80'

## Part 1:Data Preprocessing

### Upload train, validation and test files

In [5]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dev.json to dev.json
Saving test.json to test.json
Saving train.json to train.json
User uploaded file "dev.json" with length 201742 bytes
User uploaded file "test.json" with length 771364 bytes
User uploaded file "train.json" with length 1467771 bytes


### Loading the Train and Validation dataset

In [6]:
with open('/content/train.json') as f:
  input_data = json.load(f)
 
with open('/content/dev.json') as f1:
  validation_data = json.load(f1)

# with open('/content/drive/MyDrive/Data/Chata_ai/test.json') as f2:
#   test_data = json.load(f2)

# df2 = pd.DataFrame.from_dict(test_data, orient='index')
# print(df2)

df = pd.DataFrame.from_dict(input_data, orient='index')
df1 = pd.DataFrame.from_dict(validation_data, orient='index')

### Removing extra spaces (if any)

In [7]:
df['original'] = df['original'].apply(lambda x:' '.join(x.split()))
df['disfluent'] = df['disfluent'].apply(lambda x: ' '.join(x.split()))

df1['original'] = df1['original'].apply(lambda x:' '.join(x.split()))
df1['disfluent'] = df1['disfluent'].apply(lambda x: ' '.join(x.split()))

### Removing symbols in the dataset

In [8]:
punctuation = '!"#$&()%*/:;,\'<?@[]^_`{|}~'

df['original'] = df['original'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
df['disfluent'] = df['disfluent'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

df1['original'] = df1['original'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
df1['disfluent'] = df1['disfluent'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

### Tokenize with AutoTokenizer

In [9]:
# batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
# len(batch)
# encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoded_input_data = [tokenizer.tokenize(i) for i in df['disfluent']]
encoded_input_labels = [tokenizer.tokenize(i) for i in df['original']]
encoded_validation_data = [tokenizer.tokenize(i) for i in df1['disfluent']]
encoded_validation_labels = [tokenizer.tokenize(i) for i in df1['original']]

### Converting the tokens to the index numbers in BERT vocabulary

In [11]:
MAX_LEN = 64

input_data_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_input_data]
input_data_ids = pad_sequences(input_data_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

input_labels_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_input_labels]
input_labels_ids = pad_sequences(input_labels_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

validation_data_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_validation_data]
validation_data_ids = pad_sequences(validation_data_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

validation_labels_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_validation_labels]
validation_labels_ids = pad_sequences(validation_labels_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

### Creating attention masks

In [12]:
in_attention_masks = []
val_attention_masks = []

for in_seq in input_data_ids:
  in_seq_mask = [float(i>0) for i in in_seq]
  in_attention_masks.append(in_seq_mask)

for val_seq in validation_data_ids:
  val_seq_mask = [float(i>0) for i in val_seq]
  val_attention_masks.append(val_seq_mask)

### Converting train and validation data to torch tensors

In [13]:
train_inputs = input_data_ids
train_inputs = torch.tensor(train_inputs)

train_labels = input_labels_ids
train_labels = torch.tensor(train_labels)

validation_inputs = validation_data_ids
validation_inputs = torch.tensor(validation_inputs)

validation_labels = validation_labels_ids
validation_labels = torch.tensor(validation_labels)

train_masks = in_attention_masks
train_masks = torch.tensor(in_attention_masks)

validation_masks = val_attention_masks
validation_masks = torch.tensor(val_attention_masks)

### Creating iterator of the data

In [14]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### Displaying model's parameters as a list of tuples

In [15]:
model = BertModel.from_pretrained("bert-base-uncased")
model.cuda()

params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

100%|██████████| 407873900/407873900 [00:18<00:00, 22286827.20B/s]


The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (

### Fine-tuning BERT parameters

In [16]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
train_loss_set = []
epochs = 4

for _ in trange(epochs, desc="Epoch"):  
  
  # train_model = model.train()  
  # tr_loss = []
  # nb_tr_examples, nb_tr_steps = 0, 0
  # for step, batch in enumerate(train_dataloader):
  #   batch = tuple(t.to(device) for t in batch)
  #   b_input_ids, b_input_mask, b_labels = batch
  #   optimizer.zero_grad()
  #   loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)#, labels=b_labels)
  #   train_loss_set.append(loss)    
  #   loss.backward()
  #   optimizer.step()
  #   tr_loss.append(loss)
  #   nb_tr_examples += b_input_ids.size(0)
  #   nb_tr_steps += 1
  # print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION
  validation_model = model.eval()
#   eval_loss, eval_accuracy = 0, 0
#   nb_eval_steps, nb_eval_examples = 0, 0
#   for batch in validation_dataloader:
#     batch = tuple(t.to(device) for t in batch)
#     b_input_ids, b_input_mask, b_labels = batch
#     with torch.no_grad():
#       logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
#     logits = logits.detach().cpu().numpy()
#     label_ids = b_labels.to('cpu').numpy()
#     tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
#     eval_accuracy += tmp_eval_accuracy
#     nb_eval_steps += 1
#   print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

# plt.figure(figsize=(15,8))
# plt.title("Training loss")
# plt.xlabel("Batch")
# plt.ylabel("Loss")
# plt.plot(train_loss_set)
# plt.show()

t_total value of -1 results in schedule not being applied
Epoch: 100%|██████████| 4/4 [00:00<00:00, 1077.19it/s]


### Loading test dataset, preparing inputs and evaluating the model

In [17]:
with open('/content/test.json') as f2:
  test_data = json.load(f2)

df2 = pd.DataFrame.from_dict(test_data, orient='index')

df2['original'] = df2['original'].apply(lambda x:' '.join(x.split()))
df2['disfluent'] = df2['disfluent'].apply(lambda x: ' '.join(x.split()))

df2['original'] = df2['original'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
df2['disfluent'] = df2['disfluent'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# tokenize test data
encoded_test_data = [tokenizer.tokenize(i) for i in df2['disfluent']]
encoded_test_labels = [tokenizer.tokenize(i) for i in df2['original']]

output_data_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_test_data]
output_data_ids = pad_sequences(output_data_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

output_labels_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_test_labels]
output_labels_ids = pad_sequences(output_labels_ids, maxlen = MAX_LEN, dtype = "long", truncating = "post", padding = "post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in output_data_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(output_data_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(output_labels_ids)
batch_size = 32  
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Prediction on test set
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  # Move logits and labels to CPU
  # logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  # Store predictions and true labels
  predictions.append(logits)
  del logits
  true_labels.append(label_ids)

In [ ]:
# import tensorflow_hub as hub
# import tensorflow as tf

# elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
# def elmo_vectors(x):
#   embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

#   with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.tables_initializer())
#     # return average of ELMo features
#     return sess.run(tf.reduce_mean(embeddings,1))

# df = [df[i:i+100] for i in range(0,df.shape[0],100)]
# # df['original'] = [df['original'][i:i+100] for i in range(0,df['original'].shape[0],100)]
# # df['disfluent'] = df['disfluent'][i:i+100] for i in range(0,df['disfluent'].shape[0],100)]

# print(df)
# len(df)

# # list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

# # Extract ELMo embeddings
# # elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
# # elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

# # elmo_train_new = np.concatenate(elmo_train, axis = 0)
# # elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# elmo_train = [elmo_vectors(x['disfluent']) for x in df]
# elmo_train_new = np.concatenate(elmo_train, axis = 0)